# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [4]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features



In [28]:
market = pd.read_csv('../../estaticos_market.csv')
port1 = pd.read_csv('clientes2_merge.csv')

In [29]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [30]:
port1.shape

(566, 182)

In [31]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [32]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(566, 37)

In [33]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [34]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [35]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [36]:
#market_p1.head(2)

## Encoding

In [37]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [38]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [39]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [40]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [41]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
09e95c1a84049001d086470a8f320a19b076f955a89122ff8360ac2e4d6eca5d,1,11.243836,0,0,0,0,0,1,1,0,...,0,0,0,4,0,0,17,57,6,6
dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728bb62194ca0842cce3e,1,43.789041,0,0,0,0,0,0,0,0,...,0,0,19,4,0,0,0,1,4,4
16843c9ffb920170477118933798510d8d5f00f5c56c1221773356355bded321,1,16.389041,0,0,0,0,0,0,1,0,...,36,11,16,4,0,0,9,11,3,3


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [42]:
market_sample = market_encoded.sample(n=500)

In [43]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors(n_neighbors=2, metric='cosine',radius = 5)

nearest.fit(market_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2, radius=5)

Dict para a lista de vizinhos mais próximos

In [44]:
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [45]:
neighbors_list

{0: (array([[2.69455677e-08, 2.88019486e-08]]), array([[ 41, 131]])),
 1: (array([[2.64758216e-08, 2.82633498e-08]]), array([[ 41, 131]])),
 2: (array([[1.41339483e-08, 1.52487080e-08]]), array([[ 41, 131]])),
 3: (array([[2.67104932e-08, 2.85216735e-08]]), array([[ 41, 131]])),
 4: (array([[1.41171381e-08, 1.52362840e-08]]), array([[ 41, 131]])),
 5: (array([[6.87539525e-09, 8.54160531e-09]]), array([[ 41, 131]])),
 6: (array([[1.75798361e-08, 1.95011970e-08]]), array([[ 41, 131]])),
 7: (array([[2.15753214e-05, 9.49799349e-05]]), array([[ 4, 91]])),
 8: (array([[8.06446574e-06, 8.06545878e-06]]), array([[ 41, 131]])),
 9: (array([[1.32485821e-08, 1.41229032e-08]]), array([[ 41, 131]])),
 10: (array([[1.92103058e-08, 2.03927127e-08]]), array([[ 41, 131]])),
 11: (array([[0.00043899, 0.00116064]]), array([[91,  4]])),
 12: (array([[6.85946866e-09, 8.14763235e-09]]), array([[ 41, 131]])),
 13: (array([[1.19613295e-08, 1.36900159e-08]]), array([[ 41, 131]])),
 14: (array([[1.43135316e-05

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [46]:
neighbors_list

{0: (array([[2.69455677e-08, 2.88019486e-08]]), array([[ 41, 131]])),
 1: (array([[2.64758216e-08, 2.82633498e-08]]), array([[ 41, 131]])),
 2: (array([[1.41339483e-08, 1.52487080e-08]]), array([[ 41, 131]])),
 3: (array([[2.67104932e-08, 2.85216735e-08]]), array([[ 41, 131]])),
 4: (array([[1.41171381e-08, 1.52362840e-08]]), array([[ 41, 131]])),
 5: (array([[6.87539525e-09, 8.54160531e-09]]), array([[ 41, 131]])),
 6: (array([[1.75798361e-08, 1.95011970e-08]]), array([[ 41, 131]])),
 7: (array([[2.15753214e-05, 9.49799349e-05]]), array([[ 4, 91]])),
 8: (array([[8.06446574e-06, 8.06545878e-06]]), array([[ 41, 131]])),
 9: (array([[1.32485821e-08, 1.41229032e-08]]), array([[ 41, 131]])),
 10: (array([[1.92103058e-08, 2.03927127e-08]]), array([[ 41, 131]])),
 11: (array([[0.00043899, 0.00116064]]), array([[91,  4]])),
 12: (array([[6.85946866e-09, 8.14763235e-09]]), array([[ 41, 131]])),
 13: (array([[1.19613295e-08, 1.36900159e-08]]), array([[ 41, 131]])),
 14: (array([[1.43135316e-05

In [47]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

In [48]:
neighbors_distance_array[0]

2.6945567666025738e-08

Criar um dicionário com o número da linha, id e distância

In [49]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (idx, market_sample.iloc[idx].name, neighbors_distance_array[ind])

In [50]:
dicio

{0: (41,
  '8a18a5fc6e0397a7c42000266cd2d7778ab29c80c3e6b12a7477a0b411eac084',
  2.6945567666025738e-08),
 1: (131,
  'fe3fd8e6133b2028be1c67d3a020817112fb4b688d021f17a93fbf217898f344',
  2.8801948603884853e-08),
 2: (41,
  '8a18a5fc6e0397a7c42000266cd2d7778ab29c80c3e6b12a7477a0b411eac084',
  2.6475821646165798e-08),
 3: (131,
  'fe3fd8e6133b2028be1c67d3a020817112fb4b688d021f17a93fbf217898f344',
  2.8263349771684432e-08),
 4: (41,
  '8a18a5fc6e0397a7c42000266cd2d7778ab29c80c3e6b12a7477a0b411eac084',
  1.4133948345040892e-08),
 5: (131,
  'fe3fd8e6133b2028be1c67d3a020817112fb4b688d021f17a93fbf217898f344',
  1.5248707962989272e-08),
 6: (41,
  '8a18a5fc6e0397a7c42000266cd2d7778ab29c80c3e6b12a7477a0b411eac084',
  2.67104931506168e-08),
 7: (131,
  'fe3fd8e6133b2028be1c67d3a020817112fb4b688d021f17a93fbf217898f344',
  2.8521673467452047e-08),
 8: (41,
  '8a18a5fc6e0397a7c42000266cd2d7778ab29c80c3e6b12a7477a0b411eac084',
  1.4117138125158135e-08),
 9: (131,
  'fe3fd8e6133b2028be1c67d3a020817

In [51]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'row', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)
#neig_df.row.value_counts()

In [52]:
conflicts = neig_df.merge(port1, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(market, how='left',left_index=True,right_index=True)
#neig_df.shape, port1.shape, conflicts.shape, suggestion_with_conflicts.shape, 
suggestions = suggestion_with_conflicts.drop(conflicts.index)

In [53]:
suggestions

,row,distance,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,2.661408e-04,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,3.027937e-05,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,5.605841e-06,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,1.831727e-05,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,4.597806e-05,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,1.391655e-05,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,1.962781e-05,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,1.589877e-03,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0
07f76283ed177713c5dcb561733c1811a09a6238d8d71f1f32f3d4bb4c20c884,172,2.173171e-05,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,QUIMICA-PETROQUIMICA,INDUSTRIA,25.079452,> 20,...,CENTRO AMAZONENSE,MANAUS,1,3.0,42.666667,1.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",172650720.0,180536600.0


# Visualização dos resultados

## Mapa (Heat)

### Carrega arquivo com informações geográficas
Gerado por `analysis/Save_IBGE_data_for_Map_Visualization_in_Recommendation_System.ipynb`

In [54]:
coord_micro = pd.read_csv('coord_geograficas_microrregioes_brasil.csv')

In [55]:
### Importa módulo Folium
import folium
from folium import plugins
import random

### Gera lista com coordenadas das sugestões (suggestions)

In [56]:
map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in suggestions['nm_micro_regiao']:
    coord = coord_micro[coord_micro['nm_micro'] == local][['lat', 'lng']]
    if not coord.empty:
        map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.25),
                          coord['lng'].values[0]-random.uniform(0, 0.25)])
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning
print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 15


### Mostra heat map

In [57]:
mapa = folium.Map(location=[-8.788497,-53.879873],tiles='Stamen Toner',zoom_start=5)

mapa.add_child(plugins.HeatMap(map_coord))
mapa

## Mapa com marcadores
Passe o mouse sobre os marcadores para informação adicional.

In [60]:
# Para não ter repetições no mapa com marcadores, vou criar um novo df a partir de suggestions,
# deixando só uma entrada para cada id do market
unique_suggestions = pd.DataFrame()
for client in suggestions['row'].unique():
    idxmin_distance = suggestions[suggestions['row']==client]['distance'].idxmin()
    unique_suggestions = unique_suggestions.append(market.loc[idxmin_distance])

In [63]:
uniq_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(unique_suggestions[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 unique_suggestions[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        uniq_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+local[1]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 3


In [64]:
#incluir também clientes atuais
current_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(port1[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 port1[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        current_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+local[1]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Current clients without info about nm_micro:', null_count)

Current clients without info about nm_micro: 70


In [65]:
# Mostra marcadores com informação do id e ramo da empresa de sugestão.
markmapa = folium.Map(location=[-8.788497, -53.879873],
                  tiles='OpenStreetMap', zoom_start=5)

# tooltip = 'Click for info'
# clientes atuais
for mark in current_map_coord:
    folium.Marker([mark[0], mark[1]],
                 popup = mark[2],
                 tooltip = mark[2],
                 icon=folium.Icon(icon='ok', color='green')).add_to(markmapa)
    
for mark in uniq_map_coord:
    # sugestões
    folium.Marker([mark[0], mark[1]], popup=mark[2], tooltip=mark[2]).add_to(markmapa)

markmapa

## Lista ordenada por faixa de faturamento